In [1]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('drive',force_remount=True)

Mounted at drive


In [3]:
class BrDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.instr = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.instr)

    def __getitem__(self, idx):
        pc = self.instr.loc[idx]["PC"]
        label_if_taken = self.instr.loc[idx]["branchTaken"]
        br_target = self.instr.loc[idx]["branchTarget"]
        h1 = self.instr.loc[idx]["history1"]
        h2 = self.instr.loc[idx]["history2"]
        h3 = self.instr.loc[idx]["history3"]
        nbr_instr_count = self.instr.loc[idx]["nonBranchInstrCount"]
        trav_count = self.instr.loc[idx]["observedTraversCount"]
        if self.transform:
            pass

        #return torch.tensor((pc, br_target, h1,h2,h3,nbr_instr_count,trav_count)), label_if_taken
        return torch.tensor((pc, br_target, h1,h2,h3)).unsqueeze(0), label_if_taken

In [4]:
%cd /content/drive/MyDrive/'Colab Notebooks'/CS259

/content/drive/MyDrive/Colab Notebooks/CS259


In [12]:
csv_file = "SHORT_MOBILE-1.bt9.trace.csv"
dir = "/content/drive/MyDrive/'Colab Notebooks'/CS259"
branch_dataset_train = BrDataset(csv_file, dir)
train_loader = DataLoader(branch_dataset_train, batch_size=10000, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# in another file:
# branch_dataset_test = BrDataset(csv_file, dir)
# test_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                          shuffle=True, num_workers=2)
'''
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # input is a tuple of features
        # todo: should not use float64
        inputs, label = data
        #print(inputs)
        #print(label)
'''

'\nfor epoch in range(1):  # loop over the dataset multiple times\n    running_loss = 0.0\n    for i, data in enumerate(train_loader, 0):\n        # get the inputs; data is a list of [inputs, labels]\n        # input is a tuple of features\n        # todo: should not use float64\n        inputs, label = data\n        #print(inputs)\n        #print(label)\n'

In [13]:
print(len(train_loader))

1667


In [14]:
class Test(nn.Module):
  def __init__(self, input_size, hidden_size=128, num_layers=1):
    super(Test, self).__init__()
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) #lstm
    self.batch1 = nn.BatchNorm1d(hidden_size)
    #self.fc_alt = nn.Linear(input_size,hidden_size)
    self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
    self.batch2 = nn.BatchNorm1d(128)
    self.fc_2 = nn.Linear(128,256)
    self.batch3 = nn.BatchNorm1d(256)
    self.fc_3 = nn.Linear(256,128)
    self.batch4 = nn.BatchNorm1d(128)
    self.fc = nn.Linear(128, 1) #fully connected last layer

    self.relu = nn.ReLU()
    
  def forward(self,x):
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) #hidden state
    c_0 = torch.ones(self.num_layers, x.size(0), self.hidden_size).to(device) #internal state
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    #temp = self.fc_alt(x)
    out = self.relu(hn)
    out = self.fc_1(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc_2(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc_3(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc(out) #Final Output
    out = out.float()
    return out

In [15]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [16]:
learning_rate = 1e-2
epochs = 100
input_size=5

In [17]:
model = Test(input_size=input_size)
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Test(
  (lstm): LSTM(5, 128, batch_first=True)
  (batch1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_1): Linear(in_features=128, out_features=128, bias=True)
  (batch2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_2): Linear(in_features=128, out_features=256, bias=True)
  (batch3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_3): Linear(in_features=256, out_features=128, bias=True)
  (batch4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
)


In [ ]:
model.train()
for e in range(1, epochs+1):
    epoch_loss = 0
    epoch_acc = 0
    #for m in model.modules():
    #  if isinstance(m, nn.LSTM):
    #    print(m.weight_hh_l0.data)
    for i, data in tqdm(enumerate(train_loader, 0)):
      # get the inputs; data is a list of [inputs, labels]
      # input is a tuple of features
      # todo: should not use float64
      inputs, label = data
      inputs = inputs.float()
      inputs, label = inputs.to(device), label.to(device)
      optimizer.zero_grad()
        
      y_pred = model(inputs)
      loss = criterion(y_pred, label.unsqueeze(1))
      acc = binary_acc(y_pred, label.unsqueeze(1))
        
      loss.backward()
      optimizer.step()
        
      epoch_loss += loss.item()
      epoch_acc += acc.item()
      #print(y_pred,end=", ")
      #print(label.unsqueeze(1),end=", ")
      #print(acc.item())
    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

1645it [4:17:38,  9.44s/it]

In [ ]:
y_pred_list = []
model.eval()
with torch.no_grad():
  for i, data in enumerate(test_loader,0):
    inputs,label = data
    inputs = inputs.float()
    inputs, label = inputs.to(device), label.to(device)
    y_test_pred = model(inputs)
    y_test_pred = torch.sigmoid(y_test_pred)
    y_pred_tag = torch.round(y_test_pred)
    y_pred_list.append(y_pred_tag.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]